# Market order execution

For this example, let's consider in the BTC-USD pair, executing a series BUY market order against the SELL side of the resting orderbook.

- $(p_i, A_i)$: a SELL limit order, where $p_i$ is the price (as the units of USD that's equal in value as 1 unit of BTC); $A_i$ is the order's size (as units of BTC).
- $\{ (p_i, A_i) \mid i = 1, 2, \dots, N \}$: the SELL side of the resting orderbook, sorted by price-time priority. Specifically, $p_1 \leq p_2 \leq p_3 \leq \dots \leq p_N$.
- $(B_j, s_j)$: a BUY market order, where $B_j$ is the budget (as units of USD); $s_j$ is the maximum slippage (defined below).
- $\{ (B_j, s_j) \mid j = 1, 2, \dots, M \}$: the series of BUY market orders, sorted by priority.

Maximum slippage works as follows: suppose the best ask price at the time of executing market order $(B_j, s_j)$ is $p_{\mathrm{best}}$, the market order's _average execution price_ must not be worse than $p_{\mathrm{max}} = (1 + s_j) p_{\mathrm{best}}$. Note it's possible that the market order is executed against a limit order whose limit price $p_i > p_{\mathrm{max}}$, as long as the _average_ price over all limit order consumed is no greater than $p_{\mathrm{max}}$.

Note that in the onchain smart contract, $A_i$ and $B_j$ are integers, denoted in the minimum indivisible units of BTC and USD, respectively. However, here for simplicity, we use floating point numbers for all variables.

The algorithm is a function that has the following inputs and outputs:

- Inputs:
  - $\{ (p_i, A_i) \mid i = 1, 2, \dots, N \}$
  - $\{ (B_j, s_j) \mid j = 1, 2, \dots, M \}$
- Outputs:
  - $\mathtt{limitOrderBtcSold}_{\{1, 2, \dots, N\}}$ (for each limit order, the amount of BTC sold)
  - $\mathtt{limitOrderUsdBought}_{\{1, 2, \dots, N\}}$ (for each limit order, the amount of USD bought)
  - $\mathtt{marketOrderUsdSold}_{\{1, 2, \dots, M\}}$ (for each market order, the amount of USD sold)
  - $\mathtt{marketOrderBtcBought}_{\{1, 2, \dots, M\}}$ (for each market order, the amount of BTC bought)

The algorithm is as follows:

- $\mathtt{limitOrderBtcSold}_{\{1, 2, \dots, N\}} \gets [0, 0, \dots, 0]$
- $\mathtt{limitOrderUsdBought}_{\{1, 2, \dots, N\}} \gets [0, 0, \dots, 0]$
- $\mathtt{marketOrderUsdSold}_{\{1, 2, \dots, M\}} \gets [0, 0, \dots, 0]$
- $\mathtt{marketOrderBtcBought}_{\{1, 2, \dots, M\}} \gets [0, 0, \dots, 0]$
- $i \gets 1$ (index tracking the current limit order)
- $j \gets 1$ (index tracking the current market order)
- $A_{\mathrm{remain}} \gets A_i$ (track the remaining BTC amount in the current limit order)
- $B_{\mathrm{remain}} \gets B_j$ (track the remaining BTC amount in the current market order)
- $p_{\mathrm{max}} \gets (1 + s_j) p_i$ (the maximum average execution price the current market order)
- Loop:
  - First, compute how many BTC is available to buy. This is determined by the remaining amount in either the limit or the market order, whichever is smaller:
  - $A_{\mathrm{available}} \gets \mathrm{min} \left( A_{\mathrm{remain}}, \left\lfloor \frac{B_{\mathrm{remain}}}{p_i} \right\rfloor \right)$
  - If $p_i \leq p_{\mathrm{max}}$:
    - In this case, we buy as much BTC as possible.
    - $A_{\mathrm{buy}} \gets A_{\mathrm{available}}$
  - Else (that is, $p_i > p_{\mathrm{max}}$)
    - In this case, we buy up to the point where the average execution price reaches $p_{\mathrm{max}}$.
    - $A_{\mathrm{buy}} \gets \mathrm{min} \left( A_{\mathrm{available}}, \left\lfloor \frac{p_{\mathrm{max}} \mathtt{marketOrderBtcBought}_j - \mathtt{marketOrderUsdSold}_j}{p_i - p_{\mathrm{max}}} \right\rfloor \right)$
  - $\mathtt{limitOrderBtcSold}_i \mathrel{{+}{=}} A_{\mathrm{buy}}$
  - $\mathtt{limitOrderUsdBought}_i \mathrel{{+}{=}} p_i A_{\mathrm{buy}}$
  - $\mathtt{marketOrderUsdSold}_j \mathrel{{+}{=}} p_i A_{\mathrm{buy}}$
  - $\mathtt{marketOrderBtcBought}_j \mathrel{{+}{=}} A_{\mathrm{buy}}$
  - $A_{\mathrm{remain}} \mathrel{{-}{=}} A_{\mathrm{buy}}$
  - $B_{\mathrm{remain}} \mathrel{{-}{=}} p_i A_{\mathrm{buy}}$
  - If either a) $B_{\mathrm{remain}} = 0$, or b) $B_{\mathrm{remain}} \neq 0$ and $A_{\mathrm{remain}} \neq 0$
    - (a) means this market order is completely consumed; (b) means we can't execute this market any more without exceeding $p_{\mathrm{max}}$ (the price limit).
    - Either way, move on to the next market order.
    - $j \gets j + 1$
    - $B_{\mathrm{remain}} \gets B_j$
    - $p_{\mathrm{max}} \gets (1 + s_j) p_i$
  - If $A_{\mathrm{remain}} = 0$:
    - This limit order is completely consumed. Move on to the next limit order.
    - $i \gets i + 1$
    - $A_{\mathrm{remain}} \gets A_i$
- Return $\mathtt{limitOrderBtcSold}_{\{1, 2, \dots, N\}}$, $\mathtt{limitOrderUsdBought}_{\{1, 2, \dots, N\}}$, $\mathtt{marketOrderUsdSold}_{\{1, 2, \dots, M\}}$, $\mathtt{marketOrderBtcBought}_{\{1, 2, \dots, M\}}$